In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.contrib.layers import fully_connected
from math import sqrt
from scipy.stats import norm
from scipy.stats import uniform
import cmath #for complex numbers
from scipy.integrate import quad #for numerical integration
from sklearn.preprocessing import MinMaxScaler


In [2]:
#input:
#output: Call option price with strike K and Maturity T
def heston_closed(a,b,c,T,K,rho,V0,S0,r=0):
    
    def char_f(w,a,b,c,T,K,rho,V0,S0,r):
        alpha = -0.5*w*(w+complex(0,1))
        beta = a - rho*c*w*complex(0,1)
        gamma = c*c/2
        h = cmath.sqrt(beta*beta-4*alpha*gamma)
        rplus = (beta + h)/c/c
        rminus = (beta - h)/c/c
        g = rminus/rplus
        D =  rminus*(1-cmath.exp(-h*T))/(1-g*cmath.exp(-h*T))
        C = a*(rminus*T-2/c/c*cmath.log((1-g*cmath.exp(-h*T))/(1-g)))
        return cmath.exp(C*b+D*V0+complex(0,1)*w*cmath.log(S0*cmath.exp(r*T)))

    def integrand1(w):
        i = complex(0,1)
        return (cmath.exp(-i*w*cmath.log(K))*char_f(w-i,a,b,c,T,K,rho,V0,S0,r)/i/w/char_f(-i,a,b,c,T,K,rho,V0,S0,r)).real
    
    def integrand2(w):
        i = complex(0,1)
        return (cmath.exp(-i*w*cmath.log(K))*char_f(w,a,b,c,T,K,rho,V0,S0,r)/i/w).real
    
    pi1 = 0.5 + quad(integrand1,0,np.inf)[0]/np.pi
    pi2 = 0.5 + quad(integrand2,0,np.inf)[0]/np.pi
    
    return (S0*pi1 + cmath.exp(-r*T)*K*pi2).real

In [3]:
num_input_parameters = 6
num_output_parameters = 1
learning_rate = 0.1
num_steps = 1000
batch_size = 10
num_neurons = 8

scaler = MinMaxScaler()

X = tf.placeholder(tf.float32, [None, num_input_parameters])
y = tf.placeholder(tf.float32, [None, num_output_parameters])

In [4]:
#input: X,y,batch_size
#output: X_batch,y_batch: randomly chosen rows from X,y 
def next_batch_heston(batch_size,X=None,y=None):
    #for heston X contains values for a,b,c,rho,T,K and S0,V0 are kept fixed for now
    X = np.zeros((batch_size,num_input_parameters))
    y = np.zeros((batch_size,num_output_parameters))
    
    X[:,0] = uniform.rvs(size=batch_size)+1 #a
    X[:,1] = uniform.rvs(size=batch_size)+1 #b
    X[:,2] = uniform.rvs(size=batch_size) #c
    X[:,3] = uniform.rvs(size=batch_size) #rho
    X[:,4] = 10*uniform.rvs(size=batch_size)+1 #maturities T 
    X[:,5] = np.zeros(batch_size)+0.5 #strikes K
    S0 = 1
    V0 = 1
    
    for i in range(batch_size):
        y[i,0] = heston_closed(X[i,0],X[i,1],X[i,2],X[i,4],X[i,5],X[i,3],V0,S0)
    
    X = scaler.fit_transform(X)
    y = scaler.fit_transform(y)
    
    return X,y

In [5]:
#Layers
hidden1 = fully_connected(X, num_neurons, activation_fn=tf.nn.elu)
hidden2 = fully_connected(hidden1, num_neurons, activation_fn=tf.nn.elu)
hidden3 = fully_connected(hidden2, num_neurons, activation_fn=tf.nn.elu)
outputs = fully_connected(hidden3, num_output_parameters, activation_fn=None)

Instructions for updating:
Please use `layer.__call__` method instead.


In [6]:
#Loss Function
loss = tf.reduce_mean(tf.sqrt(tf.square(outputs - y)))  # MSE

In [7]:
#Optimizer
optimizer = tf.train.AdamOptimizer(learning_rate)
train = optimizer.minimize(loss)

In [8]:
init = tf.global_variables_initializer()

In [9]:
saver = tf.train.Saver()

In [12]:
with tf.Session() as sess:
    sess.run(init)
    
    for iteration in range(num_steps):
        
        X_batch,Y_batch = next_batch_heston(batch_size)
        sess.run(train,feed_dict={X: X_batch, y: Y_batch})
        
        if iteration % 200 == 0:
            
            rmse = loss.eval(feed_dict={X: X_batch, y: Y_batch})
            print(iteration, "\tRMSE:", rmse)
    
    saver.save(sess, "./models/heston_closed_nn")

0 	RMSE: 0.98573714
200 	RMSE: 0.03741647
400 	RMSE: 0.07821032
600 	RMSE: 0.06188976
800 	RMSE: 0.11691568
